# Simulating the Pilcher Gate

In [1]:
# Import packages.
import numpy as np
import qctrlvisualizer
import boulderopal as bo

In [13]:
# Set parameters.
Omega_0 = 3.5 * 2 * np.pi * 1e6  # rad/s
Delta_0 = Omega_0 * 0.377  # rad/s
cutoff_frequency = Omega_0
segment_count = 256
sample_times = np.linspace(0, duration, segment_count)
duration = 1

# Define system operators.
basis_labels = ["00", "01", "10", "11", "0r", "r0", "W", "-W", "rr"]
drive_operator = np.zeros((9, 9))
drive_operator[([1, 2], [4, 5])] = 1
drive_operator[3, 6] = np.sqrt(2)

detuning_operator = np.diag([0, 1, 1, 1, -1, -1, -1, 0, 0])

# Components for the CZ operator.
h_00 = np.zeros((9, 9))
h_00[0, 0] = 1
h_01 = np.zeros((9, 9))
h_01[1, 1] = 1
h_01[2, 2] = 1
h_11 = np.zeros((9, 9))
h_11[3, 3] = 1

In [78]:
def optimal_CZ(cost_node="infidelity", optimization_count=10, **robustness):
    graph = bo.Graph()
    
    duration_op = graph.optimizable_scalar(10e-7, 10e-5, is_lower_unbounded=True, is_upper_unbounded=True)
    duration_op.name = "duration"
    
    phase_op = graph.optimization_variable(2, 0, 2 * np.pi, is_lower_unbounded=True, is_upper_unbounded=True, name = "laser_phases")
    
    Omega = graph.complex_pwc_signal(moduli=[Omega_0,Omega_0], phases=phase_op, duration = duration, name='amplitude')
    Delta = graph.complex_pwc_signal(moduli=[Delta_0,Delta_0], phases=phase_op, duration = duration, name='detuning')

    # Create Hamiltonian.
    drive_term = graph.hermitian_part(Omega * drive_operator)
    delta_term = graph.hermitian_part(Delta * detuning_operator)
    hamiltonian = (drive_term + delta_term) * duration_op
    
    theta_s = graph.optimizable_scalar(
        lower_bound=0.0,
        upper_bound=2 * np.pi,
        is_lower_unbounded=True,
        is_upper_unbounded=True,
        name="theta_s",
    )
    
    cz_op = (
        h_00
        + graph.exp(1j * theta_s) * h_01
        + graph.exp(1j * (2 * theta_s + np.pi)) * h_11
    )
    target = graph.target(operator=cz_op)
    
    noise_list = []
    if robustness["dephasing"]:
        noise_list.append(detuning_operator / duration)
    if robustness["amplitude"]:
        noise_list.append(drive_term)
    penalty = robustness["decay"]

    
    infidelity = graph.infidelity_pwc(
        hamiltonian=hamiltonian,
        target=target,
        noise_operators=noise_list,
        name="infidelity",
    )

    unitary = graph.time_evolution_operators_pwc(
        hamiltonian=hamiltonian, sample_times=sample_times, name="unitary"
    ) 

    decay_cost = graph.sum(
        graph.abs(unitary[:, 4, 1]) ** 2
        + graph.abs(unitary[:, 5, 2]) ** 2
        + graph.abs(unitary[:, 6, 3]) ** 2
        + graph.abs(unitary[:, 7, 3]) ** 2
        + 2 * graph.abs(unitary[:, 8, 3]) ** 2
    ) * (0.25 * sample_times[1] / sample_times[-1])

    decay_cost.name = "decay cost"
    
    cost = graph.log(infidelity + penalty * decay_cost, name="cost")

    result = bo.run_optimization(
        graph=graph,
        output_node_names=[
            "theta_s",
            "amplitude",
            "detuning",
            "unitary",
            "infidelity",
            "decay cost",
            "duration",
            "laser_phases"
        ],
        cost_node_name=cost_node,
        optimization_count=optimization_count,
    )

    return result



In [79]:
result = optimal_CZ(
    cost_node="infidelity",
    dephasing=False,
    amplitude=False,
    decay=0.0,
)

Your task (action_id="1948837") is queued.
Your task (action_id="1948837") has started.
Your task (action_id="1948837") has completed.


In [66]:
print("Gate infidelity is", f"{result['output']['infidelity']['value']:.3e}")
print("Single-qubit phase is", f"{result['output']['theta_s']['value']:.3f}")
print("Pulse Duration: ", f"{result['output']['duration']['value']:.3f}")
print("Pulse Phases: ", f"{(result['output']['laser_phases']['value'])/(np.pi)}")

Gate infidelity is 1.895e-04
Single-qubit phase is 3.145
Pulse Duration:  0.000
Pulse Phases:  [ 0.76804198 -1.056578  ]
